In [ ]:
import pandas as pd
import numpy as np

In [ ]:
oe = pd.read_csv('/home/athlete_events.csv')
noc= pd.read_csv('/home/noc_regions.csv')

In [ ]:
noc.head(5)

In [ ]:
oe = oe.merge(noc,on=['NOC'])


In [ ]:
oe.sample(5)

In [ ]:
# 1. How many Olympics games have been held? List down all Olympics games held
# so far and where they were held.

oe[['Games','City']].drop_duplicates().sort_values(by='Games')

In [ ]:
# 2. Which athlete from India won the most medals in Olympics (till 2016)?

oe[(oe['Team'] == 'India') & (oe['Year'] < 2016)].groupby(['ID','Name'],dropna=False)['Medal'].count().reset_index().sort_values(by=['Medal'],ascending=False).head(3)

In [ ]:
# 3. Mention the total number of nations who participated in each Olympics game.

oe.groupby(['Games'])['NOC'].nunique().reset_index().sort_values(by='Games')

In [ ]:
# 4. Which year saw the highest and lowest num of countries participating in Olympics?

min_y = oe.groupby(['Year'])['region'].nunique().reset_index().sort_values(by=['region']).head(1)
max_y = oe.groupby(['Year'])['region'].nunique().reset_index().sort_values(by=['region'],ascending=False).head(1)
df = min_y.merge(max_y,how='cross',suffixes=('_min','_max'))
df

In [ ]:
# 5. Which nation(s) has participated in all of the Olympics games?
x = max(oe.groupby(['region'])['Games'].nunique())
oe.groupby(['region'])['Games'].nunique().reset_index().query('Games == @x')


In [ ]:
# 6. Identify the sport which was played in all summer Olympics

x = oe[oe['Games'].str.contains('Summer')].nunique()['Games']
df = oe[oe['Games'].str.contains('Summer')].groupby(['Sport'])['Games'].nunique().reset_index()
df[df['Games'] == x]


In [ ]:
# 7. Which Sports were just played only once in the Olympics?

df = oe.groupby(['Sport'])['Games'].nunique().reset_index()
df[df['Games'] == 1]

In [ ]:
# 8. Fetch the total no of sports played in each Olympics games.

oe.groupby(['Games'])['Sport'].nunique().reset_index()

In [ ]:
# 9. Fetch details of the oldest athletes to win a gold medal.

oe[oe['Medal']=='Gold'].sort_values(by=['Age'],ascending=False).head(25)

In [ ]:
# 10.Find the Ratio of male and female athletes participated in all Olympics games.

m = oe[oe['Sex']=='M'].groupby(['Games'])['Name'].nunique().reset_index()
f = oe[oe['Sex']=='F'].groupby(['Games'])['Name'].nunique().reset_index()
df = m.merge(f,on=['Games'],how='left').fillna(0)
df['m_r'] = df.apply(lambda x: f'{round(x.Name_x/(x.Name_x + x.Name_y)*100,2)}%',axis=1)
df['f_r'] = df.apply(lambda x: f'{round(x.Name_y/(x.Name_x + x.Name_y)*100,2)}%',axis=1)
df

In [ ]:
# 11.Fetch the top 5 athletes who have won the most gold medals

oe[oe['Medal'] == 'Gold'].groupby(['Name'])['Medal'].count().reset_index().sort_values(by=['Medal'],ascending=False).head(5)

In [ ]:
# 12.Fetch the top 5 athletes who have won the most medals (gold/silver/bronze).

oe[oe['Medal'].isna()==False].groupby(['Name'])['Medal'].count().reset_index().sort_values(by=['Medal'],ascending=False).head(5)

In [ ]:
# 13.Fetch the top 5 most successful countries in Olympics. Success is defined
# by no. of medals won.

oe[oe['Medal'].isna()==False].groupby(['region'])['Medal'].count().reset_index().sort_values(by=['Medal'],ascending=False).head(5)

In [ ]:
# 14.List down total gold, silver, and bronze medals won by each country.
pt = oe[oe['Medal'].isna()==False].groupby(['region','Medal'])['NOC'].count().reset_index()
pd.pivot_table(pt,index='region',columns='Medal',values='NOC').fillna(0)

In [ ]:
# 15.List down total gold, silver and bronze medals won by each country
# corresponding to each Olympics games.

pt = oe[oe['Medal'].isna()==False].groupby(['region','Games','Medal'])['NOC'].count().reset_index()
x = pd.pivot_table(pt,index=['region','Games'],columns='Medal',values='NOC').fillna(0).reset_index()
x[['Gold']].astype(int)
x

In [ ]:
# 16.Identify which country won the most gold, most silver and
# most bronze medals in each Olympics games.

pt = oe[oe['Medal'].isna()==False].groupby(['region','Games','Medal'])['NOC'].count().reset_index()
x = pd.pivot_table(pt,index=['region','Games'],columns='Medal',values='NOC').fillna(0).reset_index()
x_b = x
x_b['max_bronze'] = x_b.groupby(['Games'])['Bronze'].transform('max')
x_b = x_b[x_b['Bronze']==x_b['max_bronze']][['Games','region','max_bronze']]
x_s = x
x_s['max_silver'] = x_s.groupby(['Games'])['Silver'].transform('max')
x_s = x_s[x_s['Silver']==x_s['max_silver']][['Games','region','max_silver']]
x_g = x
x_g['max_gold'] = x_g.groupby(['Games'])['Gold'].transform('max')
x_g = x_g[x_g['Gold']==x_g['max_gold']][['Games','region','max_gold']]
df = x_b.merge(x_s,on=['Games'],suffixes=['_bronze','_silver']).merge(x_g,on=['Games']).sort_values(by=['Games'])
df

In [ ]:
# 17.Identify which country won the most gold, most silver, most bronze medals and
# the most medals in each Olympics games.

pt = oe[oe['Medal'].isna()==False].groupby(['region','Games','Medal'])['NOC'].count().reset_index()
x = pd.pivot_table(pt,index=['region','Games'],columns='Medal',values='NOC').fillna(0).reset_index()
x_b = x
x_b['max_bronze'] = x_b.groupby(['Games'])['Bronze'].transform('max')
x_b = x_b[x_b['Bronze']==x_b['max_bronze']][['Games','region','max_bronze']]
x_s = x
x_s['max_silver'] = x_s.groupby(['Games'])['Silver'].transform('max')
x_s = x_s[x_s['Silver']==x_s['max_silver']][['Games','region','max_silver']]
x_g = x
x_g['max_gold'] = x_g.groupby(['Games'])['Gold'].transform('max')
x_g = x_g[x_g['Gold']==x_g['max_gold']][['Games','region','max_gold']]
df
pt1 = pt.groupby(['Games','region',])['NOC'].sum().reset_index()
pt1['max_all'] = pt1.groupby(['Games'])['NOC'].transform('max')
pt1 = pt1[pt1['NOC'] == pt1['max_all']][['Games','region','max_all']]
df = x_b.merge(x_s,on=['Games'],suffixes=['_bronze','_silver']).merge(x_g,on=['Games']).merge(pt1,on=['Games'],suffixes=['_gold','_total'])
df.sort_values(by=['Games'])


In [ ]:
# 18.Which countries have never won gold medal but have won silver/bronze medals?

pt = oe[oe['Medal'].isna()==False].groupby(['region','Games','Medal'])['NOC'].count().reset_index()
x = pd.pivot_table(pt,index=['region','Games'],columns='Medal',values='NOC').fillna(0).reset_index()
x = x.groupby(['region'])[['Bronze','Silver','Gold']].sum().reset_index()
x[x.Gold==0]


In [ ]:
# 19.In which Sport/event, has India won the highest no. of medals.

ind_med=oe[(oe.region == 'India')&(oe.Medal.isna()==False)].groupby(['Sport','Event'])['Medal'].count().reset_index()
ind_med.sort_values(by=['Medal'],ascending=False).head(3)


In [ ]:
# 20.Break down all Olympics games where India won medal for Hockey and how
# many medals in each Olympics games.
ind_med=oe[(oe.region == 'India')&(oe.Medal.isna()==False)].groupby(['Games','Sport'])['Medal'].count().reset_index()
ind_med[ind_med.Sport=='Hockey']

